In [5]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style
import plotly.graph_objs as plotly  # for interactive plots
import plotly.graph_objects as go

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score, precision_score, recall_score
import lightgbm as lgb
import keras

import tensorflow as tf
import tensorflow_decision_forests as tfdf

import nltk
import gensim
import pickle

import warnings 
warnings.filterwarnings("ignore")

In [14]:
with open('word_embeddings_tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)
model_we = tf.keras.models.load_model("model_word_embedding.keras")
model_cnn = tf.keras.models.load_model("model_cnn_1.keras")

with open('X_train_reduced.pkl', 'rb') as handle:
    X_train_reduced = pickle.load(handle)
with open('X_val_reduced.pkl', 'rb') as handle:
    X_val_reduced = pickle.load(handle)
with open('X_test_reduced.pkl', 'rb') as handle:
    X_test_reduced = pickle.load(handle)
    
with open("y_train.pkl", 'rb') as handle:
    y_train = pickle.load(handle)
with open("y_val.pkl", 'rb') as handle:
    y_val = pickle.load(handle)
with open("y_test.pkl", 'rb') as handle:
    y_test = pickle.load(handle)

# Logistic Regression 

In [6]:
def build_lr_model(X_train, y_train, learning_rate=0.01, random_state = 1234, epochs = 100):
    """Build a TF logistic regression model using Keras.

    Args:
    learning_rate: The desired learning rate for Adam.

    Returns:
    model: A tf.keras model (graph).
    """
    tf.keras.backend.clear_session()
    np.random.seed(random_state)
    tf.random.set_seed(random_state)

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True) 
    
    # Build a model using keras.Sequential.
    model = tf.keras.Sequential()

    # Keras layers can do pre-processing. This layer will take 100x100 embeddings
    # and flatten them into vectors of size 10000.
    model.add(tf.keras.layers.Flatten())

    # This layer constructs the linear set of parameters for each input feature
    # (as well as a bias), and applies a sigmoid to the result. The result is
    # binary logistic regression.
    model.add(tf.keras.layers.Dense(
      units=1,                     # output dim (for binary classification)
      use_bias=True,               # use a bias param
      activation="sigmoid"         # apply the sigmoid function!
    ))

    # Use the Adam optimizer.
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', 
                optimizer=optimizer, 
                metrics=["accuracy"])
    history = model.fit(
        x = X_train,
        y = y_train,
        validation_split=0.2, 
        epochs=epochs, 
        callbacks=[callback])
    return model, history


## Doc2Vec

In [7]:
models_d2v = []
histories_d2v = []
learning_rates = [0.0001, 0.001, 0.01, 0.1, 1]

for lr in learning_rates: 
    print(f"\nTraining Learning Rate: {lr}")
    model, history = build_lr_model(X_train_sent_embedded, y_train, learning_rate = lr)
    models_d2v.append(model)
    histories_d2v.append(history)


Training Learning Rate: 0.0001
Epoch 1/100
3482/3482 [==============================] - 8s 2ms/step - loss: 0.6306 - accuracy: 0.6022 - val_loss: 0.5880 - val_accuracy: 0.7084
Epoch 2/100
3482/3482 [==============================] - 6s 2ms/step - loss: 0.5552 - accuracy: 0.7581 - val_loss: 0.5314 - val_accuracy: 0.7934
Epoch 3/100
3482/3482 [==============================] - 7s 2ms/step - loss: 0.5087 - accuracy: 0.8093 - val_loss: 0.4945 - val_accuracy: 0.8170
Epoch 4/100
3482/3482 [==============================] - 6s 2ms/step - loss: 0.4774 - accuracy: 0.8256 - val_loss: 0.4689 - val_accuracy: 0.8270
Epoch 5/100
3482/3482 [==============================] - 6s 2ms/step - loss: 0.4552 - accuracy: 0.8333 - val_loss: 0.4502 - val_accuracy: 0.8334
Epoch 6/100
3482/3482 [==============================] - 6s 2ms/step - loss: 0.4387 - accuracy: 0.8377 - val_loss: 0.4360 - val_accuracy: 0.8359
Epoch 7/100
3482/3482 [==============================] - 7s 2ms/step - loss: 0.4260 - accuracy: 0.

Epoch 57/100
3482/3482 [==============================] - 6s 2ms/step - loss: 0.3536 - accuracy: 0.8570 - val_loss: 0.3588 - val_accuracy: 0.8553
Epoch 58/100
3482/3482 [==============================] - 7s 2ms/step - loss: 0.3535 - accuracy: 0.8571 - val_loss: 0.3587 - val_accuracy: 0.8551
Epoch 59/100
3482/3482 [==============================] - 6s 2ms/step - loss: 0.3534 - accuracy: 0.8572 - val_loss: 0.3587 - val_accuracy: 0.8552
Epoch 60/100
3482/3482 [==============================] - 7s 2ms/step - loss: 0.3533 - accuracy: 0.8570 - val_loss: 0.3587 - val_accuracy: 0.8554
Epoch 61/100
3482/3482 [==============================] - 6s 2ms/step - loss: 0.3532 - accuracy: 0.8572 - val_loss: 0.3585 - val_accuracy: 0.8553
Epoch 62/100
3482/3482 [==============================] - 7s 2ms/step - loss: 0.3531 - accuracy: 0.8573 - val_loss: 0.3584 - val_accuracy: 0.8555
Epoch 63/100
3482/3482 [==============================] - 7s 2ms/step - loss: 0.3530 - accuracy: 0.8575 - val_loss: 0.3583 -

Epoch 13/100
3482/3482 [==============================] - 7s 2ms/step - loss: 0.3519 - accuracy: 0.8580 - val_loss: 0.3572 - val_accuracy: 0.8562
Epoch 14/100
3482/3482 [==============================] - 9s 2ms/step - loss: 0.3518 - accuracy: 0.8579 - val_loss: 0.3570 - val_accuracy: 0.8562
Epoch 15/100
3482/3482 [==============================] - 9s 3ms/step - loss: 0.3518 - accuracy: 0.8581 - val_loss: 0.3571 - val_accuracy: 0.8560
Epoch 16/100
3482/3482 [==============================] - 8s 2ms/step - loss: 0.3519 - accuracy: 0.8582 - val_loss: 0.3572 - val_accuracy: 0.8564
Epoch 17/100
3482/3482 [==============================] - 8s 2ms/step - loss: 0.3518 - accuracy: 0.8578 - val_loss: 0.3574 - val_accuracy: 0.8553
Epoch 18/100
3482/3482 [==============================] - 9s 2ms/step - loss: 0.3518 - accuracy: 0.8582 - val_loss: 0.3570 - val_accuracy: 0.8563
Epoch 19/100
3482/3482 [==============================] - 8s 2ms/step - loss: 0.3518 - accuracy: 0.8584 - val_loss: 0.3570 -

In [ ]:
# with open('models_d2v.pkl', 'wb') as handle:
#     pickle.dump(models_d2v, handle)
# with open('histories_d2v.pkl', 'wb') as handle:
#     pickle.dump(histories_d2v, handle)

In [62]:
model_d2v = models_d2v[1]

In [69]:
we_preds = np.round(model_we.predict(X_val_reduced)).flatten().astype(int)
cnn_preds = np.round(model_cnn.predict(X_val_reduced)).flatten().astype(int)
d2v_preds = np.round(model_d2v.predict(X_val_sent_embedded)).flatten().astype(int)


1451/1451 [==============================] - 2s 2ms/step


In [81]:
print(classification_report(y_val, we_preds))

              precision    recall  f1-score   support

           0       0.92      0.93      0.92     23185
           1       0.93      0.92      0.92     23230

    accuracy                           0.92     46415
   macro avg       0.92      0.92      0.92     46415
weighted avg       0.92      0.92      0.92     46415



In [82]:
print(classification_report(y_val, cnn_preds))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92     23185
           1       0.92      0.92      0.92     23230

    accuracy                           0.92     46415
   macro avg       0.92      0.92      0.92     46415
weighted avg       0.92      0.92      0.92     46415



In [80]:
print(classification_report(y_val, d2v_preds))

              precision    recall  f1-score   support

           0       0.82      0.90      0.86     23185
           1       0.89      0.80      0.84     23230

    accuracy                           0.85     46415
   macro avg       0.85      0.85      0.85     46415
weighted avg       0.85      0.85      0.85     46415



In [87]:
val_results = pd.DataFrame({"Actual": y_val,
                           "Word Embedding": we_preds,
                           "1D CNN": cnn_preds, 
                           "Sentence Embedding": d2v_preds})
d2v_correct_over_others = val_results[(val_results["Sentence Embedding"] != val_results["Word Embedding"]) & 
           (val_results["Sentence Embedding"] != val_results["1D CNN"]) & 
           (val_results["Sentence Embedding"] == val_results["Actual"])]
print("% of d2v predictions correct over other models:\n", round(d2v_correct_over_others.shape[0]/val_results.shape[0] * 100, 2)
print("Class of d2v predictions that are correct over other models:\n", d2v_correct_over_others["Actual"].value_counts(normalize = True))

0.016955725519767317
Actual
0    0.724269
1    0.275731
Name: proportion, dtype: float64


Doc2Vec implementation is worse than Word2Vec models in all criterias. 

# Boosting

Unable to incorporate tensorflow random forest/boosted trees model with tensorflow Sequential model, use LightGBM for runtime. 
Light GBM is different than traditional Gradient Boosted Trees in that it grows leaf first instead of level first, allowing it to train much faster.

References: 
- https://towardsdatascience.com/kagglers-guide-to-lightgbm-hyperparameter-tuning-with-optuna-in-2021-ed048d9838b5

- https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html#deal-with-over-fitting

- https://docs.aws.amazon.com/sagemaker/latest/dg/lightgbm-tuning.html

## Avg Word Embeddings and Doc2Vec Embeddings

Word embeddings are 2D arrays, take the average of the embeddings across each embedding index to flatten into 1D array, concatenate with 1D Doc2Vec embeddings in case DocwVec embeddings capture some information not already captured in the word embeddings

In [ ]:
#Original word embeddings
with open('X_train_word_embedded.pkl', 'rb') as handle:
    X_train_word_embedded = pickle.load(handle)
with open('X_val_word_embedded.pkl', 'rb') as handle:
    X_val_word_embedded = pickle.load(handle)
with open('X_test_word_embedded.pkl', 'rb') as handle:
    X_test_word_embedded = pickle.load(handle)

In [ ]:
#1D CNN word embeddings
with open('X_train_cnn_word_embedded.pkl', 'rb') as handle:
    X_train_cnn_word_embedded = pickle.load(handle)
with open('X_val_cnn_word_embedded.pkl', 'rb') as handle:
    X_val_cnn_word_embedded = pickle.load(handle)
with open('X_test_cnn_word_embedded.pkl', 'rb') as handle:
    X_test_cnn_word_embedded = pickle.load(handle)

In [ ]:
#Word2Vec embeddings
with open('d2v_train.pkl', 'rb') as handle:
    X_train_sent_embedded = pickle.load(handle)
with open('d2v_val.pkl', 'rb') as handle:
    X_val_sent_embedded = pickle.load(handle)
with open('d2v_test.pkl', 'rb') as handle:
    X_test_sent_embedded = pickle.load(handle)


In [ ]:
X_train_word_embedded_avg = np.array([np.mean(i, axis = 0) for i in X_train_word_embedded])
X_val_word_embedded_avg = np.array([np.mean(i, axis = 0) for i in X_val_word_embedded])
X_test_word_embedded_avg = np.array([np.mean(i, axis = 0) for i in X_test_word_embedded])

X_train_concatenated = np.concatenate((X_train_word_embedded_avg, X_train_sent_embedded), axis = 1)
X_val_concatenated = np.concatenate((X_val_word_embedded_avg, X_val_sent_embedded), axis = 1)
X_test_concatenated = np.concatenate((X_test_word_embedded_avg, X_test_sent_embedded), axis = 1)


In [ ]:
X_train_cnn_word_embedded_avg = np.array([np.mean(i, axis = 0) for i in X_train_cnn_word_embedded])
X_val_cnn_word_embedded_avg = np.array([np.mean(i, axis = 0) for i in X_val_cnn_word_embedded])
X_test_cnn_word_embedded_avg = np.array([np.mean(i, axis = 0) for i in X_test_cnn_word_embedded])

X_train_cnn_concatenated = np.concatenate((X_train_cnn_word_embedded_avg, X_train_sent_embedded), axis = 1)
X_val_cnn_concatenated = np.concatenate((X_val_cnn_word_embedded_avg, X_val_sent_embedded), axis = 1)
X_test_cnn_concatenated = np.concatenate((X_test_cnn_word_embedded_avg, X_test_sent_embedded), axis = 1)


In [ ]:
X_train_all_concatenated = np.concatenate((X_train_word_embedded_avg, X_train_cnn_word_embedded_avg, X_train_sent_embedded), axis = 1)
X_val_all_concatenated = np.concatenate((X_val_word_embedded_avg, X_val_cnn_word_embedded_avg, X_val_sent_embedded), axis = 1)
X_test_all_concatenated = np.concatenate((X_test_word_embedded_avg, X_test_cnn_word_embedded_avg, X_test_sent_embedded), axis = 1)


## Grid Search

In [37]:
n_estimators = [2000] #Set to large number, set early stopping rounds
learning_rate = [0.01] 
num_leaves = [15, 30, 50, 70] #tree grows leaf wise instead of depth wise for lightgbm, focus on tuning num leaves
min_child_samples = [100] #set number of entries needed in a leaf to prevent overfitting, set to 100 since data is large
reg_alpha = [0.1, 0.2, 0.3] #combined averaged word2vec embeddings with doc2vec embedding, so could be redundancy, set L1 regularization to drop out unnecessary features
early_stopping_rounds = [50]
random_state = [42]

param_grid = {"n_estimators": n_estimators,
              "learning_rate": learning_rate,
              "num_leaves": num_leaves,
              "min_child_samples": min_child_samples,
              "reg_alpha": reg_alpha,
              "early_stopping_rounds": early_stopping_rounds,
              "random_state": random_state}

In [38]:
lgb_model = lgb.LGBMClassifier()
grid_search = GridSearchCV(estimator = lgb_model, param_grid = param_grid, cv = 5,
                          n_jobs = -1, verbose = 2, scoring = "f1") 
grid_search.fit(X_train_concatenated, y_train, eval_set = [(X_val_concatenated, y_val)], eval_metric = "logloss")
grid_search.best_params_

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Number of positive: 69639, number of negative: 69604
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.251272 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 51000
[LightGBM] [Info] Number of data points in the train set: 139243, number of used features: 200
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500126 -> initscore=0.000503
[LightGBM] [Info] Start training from score 0.000503
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[446]	valid_0's binary_logloss: 0.204129


{'early_stopping_rounds': 50,
 'learning_rate': 0.01,
 'min_child_samples': 100,
 'n_estimators': 2000,
 'num_leaves': 50,
 'random_state': 42,
 'reg_alpha': 0.3}

In [41]:
lgb_model = grid_search.best_estimator_

In [56]:
#with open('models_lgb.pkl', 'wb') as handle:
#    pickle.dump(lgb_model, handle)

In [ ]:
lgb_model_cnn = lgb.LGBMClassifier()
grid_search_cnn = GridSearchCV(estimator = lgb_model, param_grid = param_grid, cv = 5,
                          n_jobs = -1, verbose = 2, scoring = "f1") 
grid_search_cnn.fit(X_train_cnn_concatenated, y_train, eval_set = [(X_val_cnn_concatenated, y_val)], eval_metric = "logloss")
grid_search_cnn.best_params_

## Word Embeddings Neural Network/1D CNN